# Web Scraping a la página de Wikipedia

URL: "https://es.wikipedia.org/wiki/Anexo:R%C3%A9cords_de_los_Premios_Grammy"

URL: 'https://es.wikipedia.org/wiki/Anexo:R%C3%A9cords_de_los_Premios_Grammy_Latinos#M%C3%A1s_Grammy_Latinos_ganados_en_vida.'

En este archivo se realiza Web Scraping a la Wikipedia en dos ocasiones con Beautiful Soup para extraer dos Dataframes sobre los premios Grammy y Grammy latinos que posteriormente se juntan en un solo DataFrame.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://es.wikipedia.org/wiki/Anexo:R%C3%A9cords_de_los_Premios_Grammy"
response = requests.get(url)

soup = BeautifulSoup(response.content, "lxml")
table = soup.find_all("table", class_='wikitable')[0]

records = []
for row in table.find_all("tr")[1:]:
    cells = row.find_all("td")
    if len(cells) == 3:
        artist = cells[1].text.strip()
        grammys = int(cells[2].text.strip())
        records.append({"Artista": artist, "Premios Grammy": grammys})

Grammys = pd.DataFrame(records).sort_values("Premios Grammy", ascending=False)

Grammys

,Artista,Premios Grammy
0,Beyoncé,32
1,Georg Solti,31
2,Quincy Jones,29
3,Alison Krauss,27
4,Pierre Boulez,26
5,Stevie Wonder,25
6,Vladimir Horowitz,25
7,Kanye West,24
8,U2,22
9,Henry Mancini,20


In [2]:
#Grammys.to_csv('Grammys_Winner_True.csv')

In [3]:
url = 'https://es.wikipedia.org/wiki/Anexo:R%C3%A9cords_de_los_Premios_Grammy_Latinos#M%C3%A1s_Grammy_Latinos_ganados_en_vida.'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find('table', {'class': 'wikitable'})

records = []
for th in table.find_all('th'):
    records.append(th.text.strip())

data = []
for tr in table.find_all('tr')[1:]:
    row = []
    for td in tr.find_all('td'):
        row.append(td.text.strip())
    data.append(row)

Grammy_latinos = pd.DataFrame(data, columns=records)
Grammy_latinos

,Puesto,1.º,1.º,3.º,4.º,5.º,6.º,7°,8.º,9.º,10.º,11.º,12.º
0,Artista,Juan Luis Guerra,Residente,Juanes,"Calle 13, Alejandro Sanz",Carlos Vives\nEdgar Barrera,"Ruben Blades, Carlos Álvarez, Jorge Drexler, ...","Fito Paez\nCaetano Veloso, Shakira, Rosalia","Sergio George, Thom Russo, Maná, Rafa Sardina","Café Tacuba, Banda El Recodo, Mike Couzzi, Joa...","Emilio Estefan, Daddy Yankee, Vicente Fernánde...",None,None
1,Premios en total,27,27,26,24,16,14,13,12,11,10,8,7


In [4]:
Grammy_latinos = Grammy_latinos.transpose()
Grammy_latinos.columns = Grammy_latinos.iloc[0]
Grammy_latinos.drop(Grammy_latinos.columns [0], axis=1)
Grammy_latinos = Grammy_latinos.dropna(how='any').reset_index(drop=True)
Grammy_latinos.drop(0, inplace=True)
Grammy_latinos

Puesto,Artista,Premios en total
1,Juan Luis Guerra,27
2,Residente,27
3,Juanes,26
4,"Calle 13, Alejandro Sanz",24
5,Carlos Vives\nEdgar Barrera,16
6,"Ruben Blades, Carlos Álvarez, Jorge Drexler, ...",14
7,"Fito Paez\nCaetano Veloso, Shakira, Rosalia",13
8,"Sergio George, Thom Russo, Maná, Rafa Sardina",12
9,"Café Tacuba, Banda El Recodo, Mike Couzzi, Joa...",11
10,"Emilio Estefan, Daddy Yankee, Vicente Fernánde...",10


In [5]:
Grammy_latinos = Grammy_latinos.rename(columns={'Premios en total' : 'Grammys Latinos'})
Grammy_latinos

Puesto,Artista,Grammys Latinos
1,Juan Luis Guerra,27
2,Residente,27
3,Juanes,26
4,"Calle 13, Alejandro Sanz",24
5,Carlos Vives\nEdgar Barrera,16
6,"Ruben Blades, Carlos Álvarez, Jorge Drexler, ...",14
7,"Fito Paez\nCaetano Veloso, Shakira, Rosalia",13
8,"Sergio George, Thom Russo, Maná, Rafa Sardina",12
9,"Café Tacuba, Banda El Recodo, Mike Couzzi, Joa...",11
10,"Emilio Estefan, Daddy Yankee, Vicente Fernánde...",10


In [6]:
# Separo a los artistas y crear una fila para cada uno
Grammy_latinos = Grammy_latinos.assign(Artista=Grammy_latinos['Artista'].str.split(',|\n')).explode('Artista').reset_index(drop=True)
Grammy_latinos = Grammy_latinos.head(10)

In [7]:
'''
Una vez limpios los dos dataframes, los vamos a unir para sacar el dataframe final.
'''

# Unión de los dataframes en función de la columna 'Artista'
Grammy_Winners = pd.merge(Grammys, Grammy_latinos, on='Artista', how='outer')

Grammy_Winners

,Artista,Premios Grammy,Grammys Latinos
0,Beyoncé,32.0,NaN
1,Georg Solti,31.0,NaN
2,Quincy Jones,29.0,NaN
3,Alison Krauss,27.0,NaN
4,Pierre Boulez,26.0,NaN
5,Stevie Wonder,25.0,NaN
6,Vladimir Horowitz,25.0,NaN
7,Kanye West,24.0,NaN
8,U2,22.0,NaN
9,Henry Mancini,20.0,NaN


In [8]:
# Quitamos los NaN por ceros

Grammy_Winners.fillna(0, inplace=True)

In [9]:
Grammy_Winners

,Artista,Premios Grammy,Grammys Latinos
0,Beyoncé,32.0,0
1,Georg Solti,31.0,0
2,Quincy Jones,29.0,0
3,Alison Krauss,27.0,0
4,Pierre Boulez,26.0,0
5,Stevie Wonder,25.0,0
6,Vladimir Horowitz,25.0,0
7,Kanye West,24.0,0
8,U2,22.0,0
9,Henry Mancini,20.0,0


In [10]:
Grammy_Winners['Grammys Latinos'] = Grammy_Winners['Grammys Latinos'].astype(int)
Grammy_Winners['Premios Grammy'] = Grammy_Winners['Premios Grammy'].astype(float)

In [11]:
# Hacemos el dataframe final sumando las dos columnas de premios y colocando en orden descendente

Grammy_Winners["Total Grammys"] = Grammy_Winners["Premios Grammy"] + Grammy_Winners["Grammys Latinos"]
Grammy_Winners


,Artista,Premios Grammy,Grammys Latinos,Total Grammys
0,Beyoncé,32.0,0,32.0
1,Georg Solti,31.0,0,31.0
2,Quincy Jones,29.0,0,29.0
3,Alison Krauss,27.0,0,27.0
4,Pierre Boulez,26.0,0,26.0
5,Stevie Wonder,25.0,0,25.0
6,Vladimir Horowitz,25.0,0,25.0
7,Kanye West,24.0,0,24.0
8,U2,22.0,0,22.0
9,Henry Mancini,20.0,0,20.0


In [12]:
Grammy_Winners['Total Grammys'] = Grammy_Winners['Total Grammys'].astype(int)
Grammy_Winners

,Artista,Premios Grammy,Grammys Latinos,Total Grammys
0,Beyoncé,32.0,0,32
1,Georg Solti,31.0,0,31
2,Quincy Jones,29.0,0,29
3,Alison Krauss,27.0,0,27
4,Pierre Boulez,26.0,0,26
5,Stevie Wonder,25.0,0,25
6,Vladimir Horowitz,25.0,0,25
7,Kanye West,24.0,0,24
8,U2,22.0,0,22
9,Henry Mancini,20.0,0,20


In [13]:
#Vamos a crear una columna nueva que diga si es un grammy latino o no.

# Definimos una función que devuelve True si el grammy es latino.
def grammy_latino(num):
    return num > 0

# Aplicamos la función a la columna Grammys Latinos y creamos la nueva columna
Grammy_Winners['Grammy Latino'] = Grammy_Winners['Grammys Latinos'].apply(grammy_latino)

# Ordenamos el dataframe por el número de Grammys en orden descendente
Grammy_Winners = Grammy_Winners.sort_values(by='Total Grammys', ascending=False)

Grammy_Winners

,Artista,Premios Grammy,Grammys Latinos,Total Grammys,Grammy Latino
0,Beyoncé,32.0,0,32,False
1,Georg Solti,31.0,0,31,False
2,Quincy Jones,29.0,0,29,False
11,Residente,0.0,27,27,True
10,Juan Luis Guerra,0.0,27,27,True
3,Alison Krauss,27.0,0,27,False
4,Pierre Boulez,26.0,0,26,False
12,Juanes,0.0,26,26,True
5,Stevie Wonder,25.0,0,25,False
6,Vladimir Horowitz,25.0,0,25,False


In [15]:
#Grammy_Winners.to_csv('Grammy_Winners.csv')

In [16]:
import plotly.express as px

fig = px.scatter(Grammy_Winners, x='Premios Grammy', y='Grammys Latinos', color='Grammy Latino',
                 hover_name='Artista', size_max=15, width=800, height=600,
                 labels={'Premios Grammy': 'Premios Grammy', 'Grammys Latinos': 'Premios Grammy Latinos'},
                 title='Premios Grammy vs. Premios Grammy Latinos')

fig.show()

In [19]:
import plotly.graph_objects as go

# Definir los colores de las barras
colors = ['blue' if not x else 'orange' for x in Grammy_Winners['Grammy Latino']]

# Crear la figura
fig = go.Figure()
fig.add_trace(go.Bar(
    x=Grammy_Winners['Artista'],
    y=Grammy_Winners['Total Grammys'],
    marker_color=colors
))

# Configurar el layout
fig.update_layout(
    title='Total de Grammys por artista',
    xaxis_title='Artista',
    yaxis_title='Total de Grammys'
)
fig.show()